# LSTM Model Training - Stock Prediction Capstone

This notebook trains a Long Short-Term Memory (LSTM) network to predict stock prices.

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import joblib

# Add src to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))

from utils import load_data_with_indicators
from model import create_sequences, build_lstm_model

In [ ]:
# Configuration
TICKER = 'AAPL'
START_DATE = '2015-01-01'
END_DATE = '2023-01-01'
SEQ_LENGTH = 60
EPOCHS = 20
BATCH_SIZE = 32
MODEL_PATH = '../models/lstm_model.h5'
SCALER_PATH = '../models/scaler.pkl'

## 1. Data Loading and Preprocessing

In [ ]:
df = load_data_with_indicators(TICKER, START_DATE, END_DATE)
print(f"Data shape: {df.shape}")

# Filter only numeric columns for features
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
data = df[numeric_cols].values

# Scale Data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Save scaler
os.makedirs('../models', exist_ok=True)
joblib.dump(scaler, SCALER_PATH)
print("Scaler saved.")

In [ ]:
X, y = create_sequences(scaled_data, SEQ_LENGTH)

# Split Train/Test (80/20)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

## 2. Model Training

In [ ]:
model = build_lstm_model((X_train.shape[1], X_train.shape[2]))
model.summary()

In [ ]:
history = model.fit(X_train, y_train, 
                    batch_size=BATCH_SIZE, 
                    epochs=EPOCHS, 
                    validation_data=(X_test, y_test))

model.save(MODEL_PATH)
print(f"Model saved to {MODEL_PATH}")

## 3. Evaluation

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.legend()
plt.show()

In [ ]:
# Predictions
predictions = model.predict(X_test)

# Inverse Transform - we need to construct the full shape to inverse transform
# Prediction is just the Close price (scaled)
# We created dummy rows for inverse transform in the app, let's do similar here
dummy = np.zeros(shape=(len(predictions), scaled_data.shape[1]))
dummy[:, 0] = predictions[:, 0]
predictions_rescaled = scaler.inverse_transform(dummy)[:, 0]

dummy_y = np.zeros(shape=(len(y_test), scaled_data.shape[1]))
dummy_y[:, 0] = y_test
y_test_rescaled = scaler.inverse_transform(dummy_y)[:, 0]

plt.figure(figsize=(14, 7))
plt.plot(y_test_rescaled, label='Actual Price')
plt.plot(predictions_rescaled, label='Predicted Price')
plt.title('LSTM Predictions vs Actual')
plt.legend()
plt.show()